## 빅데이터 실습

### 제주도 핫플레이스 시각화

#### 데이터 불러오기

In [72]:
# 필요라이브러리 등록
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster

In [2]:
import requests
import time
from tqdm import  tqdm

In [3]:
from matplotlib import rcParams,font_manager,rc
font_path = 'C:/Windows/Fonts/NanumGothicCoding.ttf' #나눔고딕코딩체
font_name = font_manager.FontProperties(fname=font_path).get_name() # 실제 설치된 폰트 이름 조회
rc('font',family=font_name) # 한글깨짐 현상 해결!!
rcParams['axes.unicode_minus'] = False # 한글 사용시 마이너스가 표시가 깨지는 걸 방지

In [4]:
# 불필요한 경고메시지 제거
import warnings
warnings.filterwarnings('ignore')

In [7]:
## 데이터 불러오기
rawTotal = pd.read_excel('../day04/data/1_crawling_raw.xlsx')

In [ ]:
rawTotal

In [11]:
#위치정보 확인
locCounts = rawTotal['place'].value_counts()

In [ ]:
locCounts

In [14]:
dfLocCounts = pd.DataFrame(locCounts)

In [16]:
dfLocCounts.to_excel('./data/3_location_counts.xlsx')

In [20]:
Locations = list(dfLocCounts.index)

In [29]:
Locations[3]

'Seogwipo'

#### 카카오 OPENAPI로 장소 검색
- 카카오 개발자 사이트에서 애플리케이션 등록
- 키 발급

In [ ]:
shopName = '알뜨르 비행장'
## keyword는 상호, 건물명 등으로 검색 / address는 주소로 검색
url =f'https://dapi.kakao.com/v2/local/search/keyword.json?query={shopName}'
url

header = {"Authorization" : "본인키"}

places = requests.get(url,headers=header).json()['documents']

places

In [99]:
# 장소 검색 함수 만들기
def findPlace(searchingKeyward):
    # 1. 검색API 주소 변수
    url =f'https://dapi.kakao.com/v2/local/search/keyword.json?query={searchingKeyward}'

    # 2. 헤더 구성
    header = {"Authorization" : "KakaoAK 개인키 넣기"}

    # API URL 요청
    places = requests.get(url,headers=header).json()['documents']

    # 필요 정보만 추출
    place= places[0]
    name = place['place_name']
    x = place['x']
    y = place['y']
    phoneNum = place['phone']
    addrress = place['address_name']
    return [searchingKeyward,name,phoneNum,addrress,x,y]

In [102]:
## 테스트
result = findPlace('알뜨르 비행장')
result

['알뜨르 비행장',
 '알뜨르비행장',
 '',
 '제주특별자치도 서귀포시 대정읍 상모리 1670',
 '126.271527320164',
 '33.2047161778831']

In [105]:
locationResults = []
for loc in tqdm(Locations):
    try:
        data = findPlace(loc)
        locationResults.append(data)
        time.sleep(0.3)
    except IndexError:
        pass


100%|██████████| 1028/1028 [14:30<00:00,  1.18it/s]


In [106]:
## list --> DataFram 변경
dfLocationResult = pd.DataFrame(locationResults)

In [107]:
dfLocationResult.columns = ['초기명','실제명','전화번호','주소','경도','위도']

In [ ]:
dfLocationResult

In [110]:
dfLocationResult.to_excel('./data/3_3_locations.xlsx',index=False)


#### 인스타그램 크롤링정보와 카카오API로 주소 검샐결과정보 병함


In [35]:
location_counts_df = pd.read_excel('./data/3_location_counts.xlsx',index_col=0)
locations_inform_df = pd.read_excel('./data/3_3_locations.xlsx')

In [36]:
location_data = pd.merge(locations_inform_df,location_counts_df,how='inner',left_on='초기명',right_index=True)

In [ ]:
location_data['실제명'].value_counts()

In [47]:
## 중복데이터 pivot
location_dataNew = location_data.pivot_table(values='count',index=['실제명','주소','경도','위도'],aggfunc='sum')


In [48]:
location_dataNew.reset_index(inplace=True)

In [ ]:
location_dataNew

In [52]:
location_dataNew.to_excel('./data/3_locationsFinal.xlsx',index=False)

In [54]:
## 위도, 경도가 object에서 float64로 변경
location_dataNew = location_dataNew.astype({'위도':'float64','경도':'float64'})

In [55]:
location_dataNew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   실제명     709 non-null    object 
 1   주소      709 non-null    object 
 2   경도      709 non-null    float64
 3   위도      709 non-null    float64
 4   count   709 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 27.8+ KB


#### 지도 시각화
- folium 사용

In [70]:
locHanlaMt = [33.362500,126.533694]
map = folium.Map(location=locHanlaMt,zoom_start=11)
for i in range(len(location_dataNew)):
    name = location_dataNew['실제명'][i] # DF 컬럼이름 의 행갑 i
    count = location_dataNew['count'][i]
    size= int(count)
    lat = float(location_dataNew['위도'][i])
    lng = float(location_dataNew['경도'][i])
    folium.CircleMarker(location=(lat,lng),radius=size,color='red',popup=name).add_to(map)
map

In [71]:
map.save('./data/3_jeju_hotplaces.html')

In [78]:
## 마커 클러스터

mapLocations = []
mapNames = []
for i in range(len(location_dataNew)):
    data = location_dataNew.iloc[i]
    mapLocations.append((float(data['위도']),float(data['경도'])))
    mapNames.append(data['실제명'])

locHanlaMt = [33.362500,126.533694]
map = folium.Map(location=locHanlaMt,zoom_start=11)

markerCluster = MarkerCluster(
    locations =mapLocations,popups=mapNames,
    name = '제주 핫플레이스',
    overlay=True,
    control=True
)

markerCluster.add_to(map)
folium.LayerControl().add_to(map)
map

In [79]:
map.save('./data/3_jeju_hotplaces_group.html')